In [ ]:
# === 1. IMPORTATIONS ET MONTAGE GOOGLE DRIVE ===
import os
import pandas as pd
import numpy as np
from pyproj import CRS, Transformer
from scipy.spatial import cKDTree
import gc  # Gestion mémoire
from google.colab import drive

print(f"Répertoire de travail actuel : {os.getcwd()}")
try:
    drive.mount('/content/drive')
    print("--- Google Drive monté ---")
except Exception as e:
    print(f"Erreur lors du montage de Google Drive : {e}")
    try:
        drive.mount("/content/drive", force_remount=True)
        print("--- Google Drive remonté avec succès ---")
    except Exception as e_remount:
        print(f"Erreur : {e_remount}")
        exit()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


In [ ]:
# === 2. CONFIGURATION DES CHEMINS DE FICHIERS ===
BASE_PROJECT_PATH = "/content/drive/MyDrive/Stage_CEFREM_Salma_2025/"
DATA_PATH = os.path.join(BASE_PROJECT_PATH, "Data")

INPUT_FILTERED_DATA_PATH = os.path.join(DATA_PATH, "Meteo", "InSitu", "Filtered_1970_Consolidated_Meteo_GIS_Data.csv")
STATION_METADATA_PATH = os.path.join(DATA_PATH, "Meteo", "InSitu", "Consolidated_All_Pyrenees_Stations_Metadata_StrictlyFiltered.csv")
safran_csv_path = os.path.join(DATA_PATH, "Meteo", "SAFRAN", "Données SAFRAN Pyrénées filtré_Final.csv")
output_final_df_path = os.path.join(DATA_PATH, "Combined_Data_For_Regression_Prepared.parquet")


In [ ]:
# === 3. FONCTION DE RÉDUCTION DE MÉMOIRE ===
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f"   Mémoire initiale: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if 'int' in str(col_type):
            temp = df[col].dropna()
            if not temp.empty:
                c_min, c_max = temp.min(), temp.max()
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

        elif 'float' in str(col_type):
            temp = df[col].dropna()
            if not temp.empty:
                c_min, c_max = temp.min(), temp.max()
                if c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)

        elif col_type == 'object' or pd.api.types.is_string_dtype(df[col]):
            if df[col].nunique() / len(df[col]) < 0.5 and df[col].nunique() < 50000:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f"   Mémoire après optimisation: {end_mem:.2f} MB "
              f"({100 * (start_mem - end_mem) / start_mem:.2f}% de réduction)")
    return df


In [ ]:
# === 4. CHARGEMENT DES DONNÉES STATIONS ===
df_combined_stations = pd.read_csv(INPUT_FILTERED_DATA_PATH, low_memory=False)
df_combined_stations = df_combined_stations.rename(columns={
    'NUM_POSTE': 'ID_STAT',
    'ALTI_MNT': 'ALTI_MNT',
    'SLOPE': 'Station_Slope_30m',
    'ASPECT': 'Station_Aspect_30m'
})
df_combined_stations = df_combined_stations.drop(columns=['ID_STATION', 'NOM_USUEL', 'ASPECT_RAD', 'ASPECT_COS', 'ASPECT_SIN'], errors='ignore')
df_combined_stations['DATE'] = pd.to_datetime(df_combined_stations['DATE'], errors='coerce')
df_combined_stations['ID_STAT'] = df_combined_stations['ID_STAT'].astype(str)
df_combined_stations = reduce_mem_usage(df_combined_stations, verbose=True)


In [ ]:
# === 5. MÉTADONNÉES STATIONS ===
df_metadata = pd.read_csv(STATION_METADATA_PATH, low_memory=False)
df_metadata = df_metadata.rename(columns={
    'NUM_POSTE': 'ID_STAT',
    'LAT': 'Station_Latitude',
    'LON': 'Station_Longitude',
    'ALTI': 'Station_Altitude_Obs'
})
df_metadata['ID_STAT'] = df_metadata['ID_STAT'].astype(str)
df_metadata = df_metadata[['ID_STAT', 'Station_Latitude', 'Station_Longitude', 'Station_Altitude_Obs']].drop_duplicates()
df_metadata.dropna(inplace=True)
df_metadata = reduce_mem_usage(df_metadata, verbose=True)

df_combined_stations = pd.merge(df_combined_stations, df_metadata, on='ID_STAT', how='left')
df_combined_stations.dropna(subset=['DATE', 'ID_STAT', 'Station_Latitude', 'Station_Longitude',
                                    'Station_Altitude_Obs', 'ALTI_MNT', 'Station_Slope_30m', 'Station_Aspect_30m'],
                            inplace=True)


In [ ]:
# === 6. CHARGEMENT DES DONNÉES SAFRAN ===
df_safran = pd.read_csv(safran_csv_path)
df_safran = df_safran.rename(columns={
    'PRENEI_Q': 'SAFRAN_Snowfall_daily_mm',
    'PRELIQ_Q': 'SAFRAN_Precipitation_liquid_daily_mm',
    'T_Q': 'SAFRAN_Temperature_daily_C',
    'EVAP_Q': 'SAFRAN_Evaporation_daily_mm'
})
df_safran['DATE'] = pd.to_datetime(df_safran['DATE'].astype(str).str.replace('.0', ''), format='%Y%m%d', errors='coerce')
df_safran.dropna(subset=['DATE'], inplace=True)

transformer = Transformer.from_crs(CRS("EPSG:27572"), CRS("EPSG:4326"), always_xy=True)
df_safran['SAFRAN_Longitude_grid'], df_safran['SAFRAN_Latitude_grid'] = transformer.transform(df_safran['LAMBX'].values, df_safran['LAMBY'].values)
df_safran['SAFRAN_Grid_ID'] = df_safran['SAFRAN_Longitude_grid'].round(4).astype(str) + '_' + df_safran['SAFRAN_Latitude_grid'].round(4).astype(str)
df_safran = reduce_mem_usage(df_safran, verbose=True)


In [ ]:
# === 7. COUPLAGE DES DONNÉES SAFRAN AUX STATIONS ===
tree = cKDTree(df_safran[['SAFRAN_Longitude_grid', 'SAFRAN_Latitude_grid']])
stations_unique = df_combined_stations[['ID_STAT', 'Station_Longitude', 'Station_Latitude']].drop_duplicates()
distances, indices = tree.query(stations_unique[['Station_Longitude', 'Station_Latitude']])
stations_unique['Nearest_SAFRAN_Grid_ID'] = df_safran.iloc[indices]['SAFRAN_Grid_ID'].values

df_combined_stations = pd.merge(df_combined_stations, stations_unique[['ID_STAT', 'Nearest_SAFRAN_Grid_ID']], on='ID_STAT', how='left')

df_safran = df_safran.drop_duplicates(subset=['DATE', 'SAFRAN_Grid_ID'])
df_final = pd.merge(df_combined_stations, df_safran,
                    left_on=['DATE', 'Nearest_SAFRAN_Grid_ID'],
                    right_on=['DATE', 'SAFRAN_Grid_ID'], how='left')


In [ ]:
# === 8. CRÉATION DE FEATURES SUPPLÉMENTAIRES ===
df_final.drop(columns=['Nearest_SAFRAN_Grid_ID', 'SAFRAN_Grid_ID'], inplace=True, errors='ignore')

# Colonnes essentielles
essentials = ['ID_STAT', 'DATE', 'Station_Latitude', 'Station_Longitude', 'Station_Altitude_Obs',
              'ALTI_MNT', 'Station_Slope_30m', 'Station_Aspect_30m',
              'SAFRAN_Temperature_daily_C', 'SAFRAN_Snowfall_daily_mm',
              'SAFRAN_Precipitation_liquid_daily_mm', 'SAFRAN_Evaporation_daily_mm']
df_final = df_final.dropna(subset=essentials)

# Temporelles
df_final['DATE_Year'] = df_final['DATE'].dt.year.astype(np.int16)
df_final['DATE_Month'] = df_final['DATE'].dt.month.astype(np.int8)
df_final['DATE_Day'] = df_final['DATE'].dt.day.astype(np.int8)
df_final['DATE_DayOfYear'] = df_final['DATE'].dt.dayofyear.astype(np.int16)
df_final['DATE_WeekOfYear'] = df_final['DATE'].dt.isocalendar().week.astype(np.int8)

# Orientation sin/cos
df_final['Station_Aspect_sin'] = np.sin(np.deg2rad(df_final['Station_Aspect_30m'])).astype(np.float32)
df_final['Station_Aspect_cos'] = np.cos(np.deg2rad(df_final['Station_Aspect_30m'])).astype(np.float32)


In [ ]:
# === 9. SAUVEGARDE DU FICHIER FINAL ===
for col in df_final.select_dtypes(include='category').columns:
    df_final[col] = df_final[col].astype(str)

df_final.to_parquet(output_final_df_path, index=False, engine='pyarrow', compression='snappy')
print(f" Fichier final sauvegardé à : {output_final_df_path}")
